Instalação do Ambiente virtual:

pip install virtualenv
pip install virtualenvwrapper-win

Criação do Ambiente Virtual:

python -m venv Projeto_Final\Scripts\python.exe

In [100]:
import pandas as pd, requests, sqlite3
from plyer import notification

In [101]:
# URL base da API
base_url = "https://brasilapi.com.br/api/"

In [102]:
"""
Function para exibir um alerta de erro usando Plyer
    :param mensagem: string
"""  
def alerta(mensagem):
    notification.notify(
        title= "Erro na API BrasilAPI",
        message= mensagem,
        app_name= "Projeto Final",
    )

In [103]:
"""
Function para gerar o Alerta da API
    :param base_url: string
    :param endpoint: string
    :param response.status_code: int
    
"""
def buscar_dados(endpoint):
    url = f"{base_url}{endpoint}"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        mensagem = f"Erro ao buscar dados do Endpoint {base_url}{endpoint}. Código de status: {response.status_code}"
        alerta(mensagem)  # Chama a função de alerta
        return None

In [104]:
# Tabela 1: Lista de Taxas do Brasil
conn = sqlite3.connect('ProjetoFinalDB.db')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS Taxas (
                    Nome TEXT,
                    Valor NUMERIC
                )''')

# Tabela 2: Lista de Pix do Brasil
cursor.execute('''CREATE TABLE IF NOT EXISTS Pixs (
                    ISPB NUMERIC,
                    Nome TEXT,
                    NomeReduzido TEXT,
                    Modalidade TEXT,
                    Tipo TEXT
                )''')

# Tabela 3: Lista dos CPTECs
cursor.execute('''CREATE TABLE IF NOT EXISTS CPTECS (
                    Nome TEXT,
                    Estado TEXT,
                    Id NUMERIC
                )''')
conn.close()

In [105]:
# Tabela 1: Lista de Taxas do Brasil

taxas = buscar_dados("taxas/v1")

if taxas:
    
    # Criar um DataFrame com os atributos
    nome = [taxa['nome'] for taxa in taxas]
    valor = [taxa['valor'] for taxa in taxas]

    df_taxas = pd.DataFrame({
        "Nome": nome,
        "Valor": valor,
    })
    
    df_taxas['Valor'] = df_taxas['Valor'].astype(float) #Conversão de tipo de coluna
    df_taxas['Nome'] = df_taxas['Nome'].astype(str)
    df_taxas.fillna(0, inplace=True) #Tratamento de valores ausentes
    

In [106]:
# Tabela 2: Lista de PIX do Brasil

pixs = buscar_dados("pix/v1/participants")
if pixs:

    # Criar um DataFrame com os atributos
    ispb = [pix['ispb'] for pix in pixs]
    nome = [pix['nome'] for pix in pixs]
    nome_reduzido = [pix['nome_reduzido'] for pix in pixs]
    modalidade_participacao = [pix['modalidade_participacao'] for pix in pixs]
    tipo_participacao = [pix['tipo_participacao'] for pix in pixs]

    df_pixs = pd.DataFrame({
        "ISPB": ispb,
        "Nome": nome,
        "NomeReduzido": nome_reduzido,
        "Modalidade": modalidade_participacao,
        "Tipo": tipo_participacao
    })
    
    df_pixs['ISPB'] = df_pixs['ISPB'].astype(int) #Conversão de tipo de coluna
    df_pixs['Nome'] = df_pixs['Nome'].astype(str) 
    df_pixs['NomeReduzido'] = df_pixs['NomeReduzido'].astype(str) 
    df_pixs['Modalidade'] = df_pixs['Modalidade'].astype(str)
    df_pixs['Tipo'] = df_pixs['Tipo'].astype(str)
    df_pixs.fillna(0, inplace=True) #Tratamento de valores ausentes
    df_pixs = df_pixs[df_pixs['Tipo'] == 'IDRT'] # Aplicar o filtro para selecionar apenas 'IDRT'


In [107]:
# Tabela 3: Lista CPTECs

cptecs = buscar_dados("cptec/v1/cidade")
if cptecs:

    nome = [cptec['nome'] for cptec in cptecs]
    estado = [cptec['estado'] for cptec in cptecs]
    id = [cptec['id'] for cptec in cptecs]

    df_cptec = pd.DataFrame({
        'Nome': nome,
        'Estado': estado,
        'Id': id,
    })
    
    df_cptec['Nome'] = df_cptec['Nome'].astype(str) #Conversão de tipo de coluna
    df_cptec['Estado'] = df_cptec['Estado'].astype(str)
    df_cptec['Id'] = df_cptec['Id'].astype(int) 
    df_cptec.fillna(0, inplace=True) #Tratamento de valores ausentes
    estados_nordeste = ['AL', 'BH', 'CE', 'MA', 'PB', 'PE', 'PI', 'NA', 'SG'] # Lista de estados do Nordeste
    df_cptec_nordeste = df_cptec[df_cptec['Estado'].isin(estados_nordeste)]# Aplicar o filtro para pegar apenas os estados do Nordeste


In [108]:
conn = sqlite3.connect('ProjetoFinalDB.db')

dataframes = {
    'df_taxas': 'Taxas',
    'df_pixs': 'Pixs',
    'df_cptec_nordeste': 'CPTECS'
}

[locals()[df].to_sql(table, conn, if_exists='replace', index=False) for df, table in dataframes.items() if df in locals()]

query = "SELECT * FROM Taxas"
dft = pd.read_sql(query, conn)

query = "SELECT * FROM Pixs"
dfp = pd.read_sql(query, conn)

query = "SELECT * FROM CPTECS"
dfc = pd.read_sql(query, conn)


# Exibe as 5 primeiras linhas do DataFrame
display(dft.head(5))
display(dfp.head(5))
display(dfc.head(5))

conn.commit()
conn.close()

,Nome,Valor
0,Selic,12.75
1,CDI,12.65
2,IPCA,5.19


,ISPB,Nome,NomeReduzido,Modalidade,Tipo
0,68987,COOPERATIVA DE CRÉDITO ARACREDI LTDA. - SICOOB...,CC ARACREDI LTDA.,PDCT,IDRT
1,75847,COOPERATIVA DE CRÉDITO UNICRED CENTRO-SUL LTDA...,CC UNICRED CENTRO-SUL LTDA - UNICRED CENTRO-SUL,PDCT,IDRT
2,106180,CENTRAL DAS COOPERATIVAS DE CRÉDITO DOS ESTADO...,"CCC DOS EST DE MT, MS E MUN DE CACOAL/RO",PDCT,IDRT
3,259231,SICOOB UNIMAIS METROPOLITANA - COOPERATIVA DE ...,CCLA SICOOB UNIMAIS METROPOLITANA,PDCT,IDRT
4,309024,CENTRAL DAS COOPERATIVAS DE ECONOMIA E CREDITO...,CCC ESTADO MG - CECREMGE,PDCT,IDRT


,Nome,Estado,Id
0,São Benedito,CE,4750
1,São Benedito do Rio Preto,MA,4751
2,São Benedito do Sul,PE,4752
3,São Bento,MA,4753
4,São Bento,PB,4754


In [115]:
conn = sqlite3.connect('ProjetoFinalDB.db')
"""
Junção das tabelas Taxas e Pixs

"""

resultado = pd.merge(df_cptec_nordeste, df_taxas, how="left", on=["Nome"])
resultado.fillna(0, inplace=True)
resultado.to_sql('TaxasCidade', conn, if_exists='replace', index=False)
display(resultado.head(5))

conn.close()

,Nome,Estado,Id,Valor
0,São Benedito,CE,4750,0.0
1,São Benedito do Rio Preto,MA,4751,0.0
2,São Benedito do Sul,PE,4752,0.0
3,São Bento,MA,4753,0.0
4,São Bento,PB,4754,0.0
